<a href="https://colab.research.google.com/github/santiagoprado12/Tesis/blob/main/pre_procesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
import geopandas
import datetime 
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go

In [ ]:
# make sure to install these packages before running:
#!pip install pandas
!pip install sodapy
##
##
import pandas as pd
from sodapy import Socrata
 
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)
 
# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")
 
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gt2j-8ykr",limit=2200000) 
 
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

################################################################################

url = "https://opendata.arcgis.com/datasets/782122624f364fbdbd7e287b96c4a358_6.geojson"
df = geopandas.read_file(url)

In [ ]:
results_df.head()

,fecha_reporte_web,id_de_caso,fecha_de_notificaci_n,departamento,departamento_nom,ciudad_municipio,ciudad_municipio_nom,edad,unidad_medida,sexo,fuente_tipo_contagio,ubicacion,estado,pais_viajo_1_cod,pais_viajo_1_nom,recuperado,fecha_inicio_sintomas,fecha_diagnostico,fecha_recuperado,tipo_recuperacion,per_etn_,nom_grupo_,fecha_muerte
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380,ITALIA,Recuperado,27/2/2020 0:00:00,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6,NaN,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,4/3/2020 0:00:00,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5,NaN,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,29/2/2020 0:00:00,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6,NaN,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN


In [ ]:
results_df["estado"].unique()

array(['Leve', 'Fallecido', 'N/A', 'Moderado', 'Grave', 'moderado',
       'leve', 'LEVE'], dtype=object)

In [ ]:
results_df[pd.isnull(results_df["tipo_recuperacion"]) & pd.notnull(results_df["fecha_muerte"])]

,fecha_reporte_web,id_de_caso,fecha_de_notificaci_n,departamento,departamento_nom,ciudad_municipio,ciudad_municipio_nom,edad,unidad_medida,sexo,fuente_tipo_contagio,ubicacion,estado,pais_viajo_1_cod,pais_viajo_1_nom,recuperado,fecha_inicio_sintomas,fecha_diagnostico,fecha_recuperado,tipo_recuperacion,per_etn_,nom_grupo_,fecha_muerte
151,20/3/2020 0:00:00,152,18/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,65,1,M,Importado,Fallecido,Fallecido,840,ESTADOS UNIDOS DE AMÉRICA,Fallecido,10/3/2020 0:00:00,20/3/2020 0:00:00,NaN,NaN,6,NaN,5/4/2020 0:00:00
152,20/3/2020 0:00:00,153,18/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,53,1,F,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,18/3/2020 0:00:00,20/3/2020 0:00:00,NaN,NaN,6,NaN,6/4/2020 0:00:00
156,20/3/2020 0:00:00,157,20/3/2020 0:00:00,47001,STA MARTA D.E.,47001,SANTA MARTA,88,1,M,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,12/3/2020 0:00:00,20/3/2020 0:00:00,NaN,NaN,6,NaN,23/3/2020 0:00:00
187,21/3/2020 0:00:00,188,17/3/2020 0:00:00,76,VALLE,76892,YUMBO,70,1,F,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,8/3/2020 0:00:00,21/3/2020 0:00:00,NaN,NaN,6,NaN,22/3/2020 0:00:00
196,16/3/2020 0:00:00,197,13/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,58,1,M,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,6/3/2020 0:00:00,16/3/2020 0:00:00,NaN,NaN,6,NaN,16/3/2020 0:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193855,14/2/2021 0:00:00,2193896,1/2/2021 0:00:00,68,SANTANDER,68081,BARRANCABERMEJA,80,1,F,En estudio,Fallecido,Fallecido,NaN,NaN,Fallecido,27/1/2021 0:00:00,12/2/2021 0:00:00,NaN,NaN,NaN,NaN,9/2/2021 0:00:00
2194372,14/2/2021 0:00:00,2194413,2/2/2021 0:00:00,73,TOLIMA,73217,COYAIMA,89,1,M,En estudio,Fallecido,Fallecido,NaN,NaN,Fallecido,30/1/2021 0:00:00,13/2/2021 0:00:00,NaN,NaN,NaN,NaN,12/2/2021 0:00:00
2194557,14/2/2021 0:00:00,2194598,26/12/2020 0:00:00,8,ATLANTICO,8433,MALAMBO,19,1,F,En estudio,N/A,N/A,NaN,NaN,N/A,22/12/2020 0:00:00,13/2/2021 0:00:00,NaN,NaN,NaN,NaN,26/12/2020 0:00:00
2194664,14/2/2021 0:00:00,2194705,1/2/2021 0:00:00,19,CAUCA,19532,PATIA (EL BORDO),84,1,M,En estudio,Fallecido,Fallecido,NaN,NaN,Fallecido,27/1/2021 0:00:00,12/2/2021 0:00:00,NaN,NaN,NaN,NaN,12/2/2021 0:00:00


In [ ]:
diagnosticados=results_df[pd.notnull(results_df["fecha_de_notificaci_n"])]["fecha_de_notificaci_n"]
diagnosticados_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in diagnosticados]#Creando una nueva columna formato datetime
df_diagnosticados_date_fmat=pd.DataFrame({'diagnosticados': diagnosticados_date_fmat})

In [ ]:
muertes=results_df[(pd.notnull(results_df["fecha_muerte"])) & ((results_df["recuperado"]=='fallecido') | (results_df["tipo_recuperacion"]=='Tiempo'))]["fecha_muerte"]
muertes_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in muertes]#Creando una nueva columna formato datetime
df_muertes_date_fmat=pd.DataFrame({'muertes': muertes_date_fmat})

In [ ]:
#recuperados=results_df[(pd.notnull(results_df["fecha_recuperado"]))]["fecha_recuperado"]
recuperados=results_df[results_df["recuperado"]=="Recuperado"]["fecha_recuperado"]
recuperados_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in recuperados]#Creando una nueva columna formato datetime
df_recuperados_date_fmat=pd.DataFrame({'recuperados': recuperados_date_fmat})

In [ ]:
fecha_mayor=str(results_df["fecha_de_notificaci_n"].unique()[-1])
fecha_mayor1=datetime.datetime.strptime(str(fecha_mayor),"%d/%m/%Y %H:%M:%S")

In [ ]:
c=str(results_df["fecha_reporte_web"][0])
c1=datetime.datetime.strptime(c,"%d/%m/%Y %H:%M:%S")
c2=[c1]
i=1
while c2[-1] != fecha_mayor1:
    c2.append(c2[0]+datetime.timedelta(days=i))
    i+=1

In [ ]:
casos_diarios={}
casos_diarios["fechas"]=c2
casos_diarios_DF=pd.DataFrame(data=casos_diarios)

In [ ]:
casos_diarios_DF.tail()

,fechas
342,2021-02-11
343,2021-02-12
344,2021-02-13
345,2021-02-14
346,2021-02-15


In [ ]:
fecha_Actu=[f.split('+')[0] for f in df["FECHA_ACTUALIZACION"]]
FECHA_ACTUALIZACION_DATE_FMAT=[datetime.datetime.strptime(str(x),"%Y-%m-%dT%H:%M:%S") for x in fecha_Actu]#Creando una nueva columna formato datetime
df.tail()

,OBJECTID,NUEVOS_CASOS,TOTAL_CASOS,TOTAL_MUERTES,TOTAL_RECUPERADOS,FECHA_ACTUALIZACION,GlobalID,NUEVOS_MUERTOS,NUEVO_RECUPERADOS,geometry
342,894,6294,2179641,56983,2065209,2021-02-11T05:00:00+00:00,adea36cc-ce8d-4947-bd3d-52d86fe94f2d,250,9741,None
343,895,5528,2185169,57196,2072793,2021-02-12T05:00:00+00:00,07fc282f-7f51-4ffd-9c6b-9528eef802df,213,7584,None
344,896,4947,2190116,57425,2078391,2021-02-13T05:00:00+00:00,6afbf3d0-bbfd-4ba3-8cc8-548c7054c9a2,229,5598,None
345,897,4923,2195039,57605,2083551,2021-02-14T05:00:00+00:00,3aa97797-a92d-480b-9ec9-efc4afe7426e,180,5160,None
346,898,3510,2198549,57786,2090467,2021-02-15T05:00:00+00:00,05405380-7cfd-4cca-9830-8da96255b2c2,181,6916,None


In [ ]:
casos_diarios_DF["casos por dia"]=list(map(lambda f: (df_diagnosticados_date_fmat[df_diagnosticados_date_fmat['diagnosticados']==f].shape[0]), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["fallecidos por dia"]=list(map(lambda f: (df_muertes_date_fmat[df_muertes_date_fmat['muertes']==f].shape[0]), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["recuperados por dia"]=list(map(lambda f: (df_recuperados_date_fmat[df_recuperados_date_fmat['recuperados']==f].shape[0]), casos_diarios_DF["fechas"]))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["casos por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVOS_CASOS"],
    name="base de datos"
))

fig.update_layout(title="Casos nuevos por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["fallecidos por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVOS_MUERTOS"],
    name="base de datos"
))

fig.update_layout(title="Muertos por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["recuperados por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVO_RECUPERADOS"],
    name="base de datos"
))
fig.update_layout(title="Recuperados por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
len(results_df[(results_df["recuperado"]=="Recuperado") & (results_df["tipo_recuperacion"]=="Tiempo")]["fecha_recuperado"])/len(results_df[results_df["recuperado"]=="Recuperado"]["fecha_recuperado"])

0.8882340644458869

In [ ]:
sum(casos_diarios_DF["fallecidos por dia"])

57786

In [ ]:
sum(df["NUEVOS_MUERTOS"])

57785

In [ ]:
sum(casos_diarios_DF["recuperados por dia"])

2090467

In [ ]:
listaf = casos_diarios_DF["fallecidos por dia"].rolling(len(casos_diarios_DF["fallecidos por dia"]), min_periods=1).mean()
listar = casos_diarios_DF["recuperados por dia"].rolling(len(casos_diarios_DF["recuperados por dia"]), min_periods=1).mean()
listac = casos_diarios_DF["casos por dia"].rolling(len(casos_diarios_DF["recuperados por dia"]), min_periods=1).mean()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=listaf,
    name="Preprocesamiento propio"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=listar,
    name="Preprocesamiento propio"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=listac,
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["TOTAL_RECUPERADOS"],
    name="base de datos"
))

fig.update_layout(title="Recuperados por dia")

fig.update_layout(showlegend=True)

fig.show()